In [1]:
import pandas as pd

In [2]:
import pickle

In [4]:
import math

In [7]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow


In [8]:

mlflow.set_tracking_uri("http://localhost:3000")
mlflow.set_experiment("nyc-taxi-experiment")

2025/05/31 18:31:43 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1748716303147, experiment_id='1', last_update_time=1748716303147, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [12]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [13]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet')

In [14]:
categorical = ['PU_DO'] #['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
import xgboost as xgb

In [18]:
from pathlib import Path

In [19]:
models_folder = Path("models")
models_folder.mkdir(exist_ok=True)

In [20]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params ={
    'learning_rate': 0.0575188576815762,
    'max_depth': 33,
    'min_child_weight': 3.0654071229380224,
    'objective': 'reg:linear',
    'reg_alpha': 0.006897613262011037,
    'reg_lambda': 0.06078079888314963,
    'seed': 42
    }
    
    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = (root_mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") 

/home/azureuser/anaconda3/envs/pipelines/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:46:13] WARNING: /croot/xgboost-split_1748329871263/work/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.96670
[1]	validation-rmse:8.63902
[2]	validation-rmse:8.33663
[3]	validation-rmse:8.05902
[4]	validation-rmse:7.80307
[5]	validation-rmse:7.56807
[6]	validation-rmse:7.35316
[7]	validation-rmse:7.15612
[8]	validation-rmse:6.97645
[9]	validation-rmse:6.81326
[10]	validation-rmse:6.66320
[11]	validation-rmse:6.52688
[12]	validation-rmse:6.40324
[13]	validation-rmse:6.29057
[14]	validation-rmse:6.18862
[15]	validation-rmse:6.09632
[16]	validation-rmse:6.01166
[17]	validation-rmse:5.93546
[18]	validation-rmse:5.86656
[19]	validation-rmse:5.80403
[20]	validation-rmse:5.74695
[21]	validation-rmse:5.69576
[22]	validation-rmse:5.64936
[23]	validation-rmse:5.60702
[24]	validation-rmse:5.56908
[25]	validation-rmse:5.53528
[26]	validation-rmse:5.50445
[27]	validation-rmse:5.47642
[28]	validation-rmse:5.45117
[29]	validation-rmse:5.42878
[30]	validation-rmse:5.40821
[31]	validation-rmse:5.38934
[32]	validation-rmse:5.37190
[33]	validation-rmse:5.35629
[34]	validation-rmse:5.3

/home/azureuser/anaconda3/envs/pipelines/lib/python3.13/site-packages/xgboost/core.py:158: UserWarning: [18:49:08] WARNING: /croot/xgboost-split_1748329871263/work/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/31 18:49:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run capable-mule-362 at: http://localhost:3000/#/experiments/1/runs/639fafaae34440d8a56460589d841b39
🧪 View experiment at: http://localhost:3000/#/experiments/1
